## Combine CSV script

In [1]:
import pandas as pd
import os

# Combine Asteroid CSVs into a single Full.txt file

In [18]:
# Step 1: Get all CSV files in the current directory that start with 'LVL2_Asteroids'
def get_csv_files(path):
    csv_files = [f'{path}/{f}' for f in os.listdir(path) if f.startswith('LVL2_Asteroids') and f.endswith('.csv') and 'Seg' not in f]
    return csv_files

In [19]:
# Step 2: Combine all CSV files into one dataframe
def combine_csv_files(csv_files):
    combined_df = pd.DataFrame()  # Empty DataFrame to hold all data

    for file in csv_files:
        df = pd.read_csv(file)  # Read each CSV file
        combined_df = pd.concat([combined_df, df], ignore_index=True)  # Append to the combined dataframe

    return combined_df

In [20]:
# Step 3: Remove duplicate rows based on the 'Observation' column
def remove_duplicates(combined_df):
    # Drop duplicates based on the 'Observation' column
    combined_df = combined_df.drop_duplicates(subset='Observation', keep='first')
    return combined_df

In [21]:
# Step 4: Save the combined DataFrame to a new CSV file
def save_combined_csv(combined_df, output_file):
    combined_df.to_csv(output_file, index=False)  # Save without index

In [22]:
# Main function to execute the script
def main(path):
    csv_files = get_csv_files(path)  # Get all CSV files starting with 'LVL2_Asteroids'

    if csv_files:  # Check if any matching CSV files are found
        combined_df = combine_csv_files(csv_files)  # Combine all CSV files
        combined_df = remove_duplicates(combined_df)  # Remove duplicate rows based on 'Observation'
        combined_df = combined_df.sort_values(by=['Observation']) 
        save_combined_csv(combined_df, 'LVL2_Full.csv')  # Save the result to 'LVL2_Full.csv'

        print(f"Combined CSV file saved as 'LVL3_Full.csv'.")
    else:
        print("No matching CSV files found.")

main('./')

Combined CSV file saved as 'LVL3_Full.csv'.


## Generate new CSV containing only the asteroid rows

In [25]:
# Step 1: Read the original CSV file ('CSV1')
def filter_asteroids(csv_file):
    df = pd.read_csv(csv_file)  # Read the CSV into a DataFrame
    
    # Step 2: Filter rows where the 'Asteroids' column contains '['
    filtered_df = df[df['Asteroids'].str.contains(r'\[', na=False)]# & ~df['Asteroids'].str.contains(r'\[', na=False)]  # Use regex to match '[' in 'Asteroids' column

    return filtered_df

# Step 3: Save the filtered rows to a new CSV file ('CSV2')
def save_filtered_csv(filtered_df, output_file):
    filtered_df.to_csv(output_file, index=False)  # Save without index

# Main function to execute the script
def main():
    input_csv = 'LVL2_Full.csv'  # Replace with your input CSV filename
    output_csv = 'LVL2_Only_Asteroids.csv'  # Output CSV filename

    filtered_df = filter_asteroids(input_csv)  # Get the filtered DataFrame
    save_filtered_csv(filtered_df, output_csv)  # Save it to 'CSV2.csv'

    print(f"Filtered CSV saved as '{output_csv}'.")


main()

Filtered CSV saved as 'LVL2_Only_Asteroids_NONMOVING.csv'.


# Compare the new method for combining csvs to the old output  (looking at rows that are different/ dont exist in both)

In [10]:
import pandas as pd

# Step 1: Load both CSV files into dataframes
def load_csvs(csv_file1, csv_file2):
    df1 = pd.read_csv(csv_file1)  # CSV 1 with 'Observation ID'
    df2 = pd.read_csv(csv_file2)  # CSV 2 with 'Observation' and 'Moving'
    return df1, df2

# Step 2: Find rows where 'Observation' in CSV 2 is not present in 'Observation ID' in CSV 1
def find_rows_in_csv2_not_in_csv1(df1, df2):
    return df2[~df2['Observation'].isin(df1['Observation ID'])]

# Step 3: Split the rows into two DataFrames based on the 'Moving' column ('Yes' and 'No')
def split_by_moving(df):
    df_moving_yes = df[df['Moving'].str.lower() == 'yes']
    df_moving_no = df[df['Moving'].str.lower() == 'no']
    return df_moving_yes, df_moving_no

# Main function to execute the script
def main():
    csv_file1 = 'Previous/LVL2_Asteroids.csv'  # Replace with the path to your first CSV (with 'Observation ID')
    csv_file2 = 'LVL2_Full.csv'  # Replace with the path to your second CSV (with 'Observation' and 'Moving')

    # Load both CSVs
    df1, df2 = load_csvs(csv_file1, csv_file2)

    # Find rows in CSV 2 that are not in CSV 1 based on 'Observation' / 'Observation ID'
    df2_not_in_df1 = find_rows_in_csv2_not_in_csv1(df1, df2)

    # Split the rows based on the 'Moving' column ('Yes' and 'No')
    df_moving_yes, df_moving_no = split_by_moving(df2_not_in_df1)

    # Output the resulting DataFrames
    print(f"Rows where 'Moving' is 'Yes' in CSV 2 but not in CSV 1 -{len(df_moving_yes)}-:")
    print(df_moving_yes)
    

    print(f"\nRows where 'Moving' is 'No' in CSV 2 but not in CSV 1 -{len(df_moving_no)}:")
    print(df_moving_no)

    return df_moving_yes, df_moving_no


df_moving_yes, df_moving_no = main()

print(f"\n Total new rows = {len(df_moving_yes) + len(df_moving_no)}")

Rows where 'Moving' is 'Yes' in CSV 2 but not in CSV 1 -109-:
      Proposal                    Observation Instrument  Filter Moving  \
0         1022   jw01022-o017_t001_miri_f770w       MIRI   F770W    Yes   
1         1022   jw01022-o018_t001_miri_f770w       MIRI   F770W    Yes   
2         1022   jw01022-o019_t001_miri_f770w       MIRI   F770W    Yes   
3         1022   jw01022-o020_t001_miri_f770w       MIRI   F770W    Yes   
4         1022   jw01022-o021_t001_miri_f770w       MIRI   F770W    Yes   
...        ...                            ...        ...     ...    ...   
1985      1897  jw01897-o003_t001_miri_f1130w       MIRI  F1130W    Yes   
1986      1897   jw01897-o003_t001_miri_f770w       MIRI   F770W    Yes   
1987      1897  jw01897-o004_t002_miri_f1000w       MIRI  F1000W    Yes   
1988      1897  jw01897-o004_t002_miri_f1130w       MIRI  F1130W    Yes   
1989      1897   jw01897-o004_t002_miri_f770w       MIRI   F770W    Yes   

                                     

In [12]:
import pandas as pd

# Step 1: Load both CSV files into dataframes
def load_csvs(csv_file1, csv_file2):
    df1 = pd.read_csv(csv_file1)  # CSV 1 with 'Observation ID'
    df2 = pd.read_csv(csv_file2)  # CSV 2 with 'Observation'
    return df1, df2

# Step 2: Find all rows in CSV 2 where 'Observation' is not present in 'Observation ID' in CSV 1
def find_rows_in_csv2_not_in_csv1(df1, df2):
    return df2[~df2['Observation'].isin(df1['Observation ID'])]

# Main function to execute the script
def main():
    csv_file1 = 'Previous/LVL2_Asteroids.csv'  # Replace with the path to your first CSV (with 'Observation ID')
    csv_file2 = 'LVL2_Full.csv'  # Replace with the path to your second CSV (with 'Observation')

    # Load both CSVs
    df1, df2 = load_csvs(csv_file1, csv_file2)

    # Find rows in CSV 2 that are not in CSV 1 based on 'Observation' / 'Observation ID'
    df2_not_in_df1 = find_rows_in_csv2_not_in_csv1(df1, df2)

    # Output the resulting DataFrame
    print(f"Rows from CSV 2 where 'Observation' is not present in CSV 1 -{len(df2_not_in_df1)}:-")
    pd.set_option('display.max_rows', None)
    
    #print(df2_not_in_df1)

    return df2_not_in_df1


df2_not_in_df1 = main()

Rows from CSV 2 where 'Observation' is not present in CSV 1 -486:-


## New proposals

In [37]:
import pandas as pd

# Step 1: Load both CSV files into dataframes
def load_csvs(csv_file1, csv_file2):
    df1 = pd.read_csv(csv_file1)  # CSV 1 with 'Proposal ID'
    df2 = pd.read_csv(csv_file2)  # CSV 2 with 'Proposals'
    return df1, df2

# Step 2: Find all values in 'Proposals' from CSV 2 that are not present in 'Proposal ID' from CSV 1
def find_unique_proposals(df1, df2):
    return df2[~df2['Proposal'].isin(df1['Proposal ID'])]

# Main function to execute the script
def main():
    csv_file1 = 'Previous/LVL2_Asteroids.csv'  # Replace with the path to your first CSV (with 'Proposal ID')
    csv_file2 = 'LVL2_Full.csv'  # Replace with the path to your second CSV (with 'Proposals')

    # Load both CSVs
    df1, df2 = load_csvs(csv_file1, csv_file2)

    # Find values in 'Proposals' (CSV 2) that are not in 'Proposal ID' (CSV 1)
    df_unique_proposals = find_unique_proposals(df1, df2)

    # Output the resulting DataFrame containing only the unique 'Proposals'
    print("Proposals in CSV 2 that are not present in Proposal ID of CSV 1:")
    print(df_unique_proposals['Proposal'].unique())

    return df_unique_proposals

df_unique_proposals = main()


Proposals in CSV 2 that are not present in Proposal ID of CSV 1:
[1022 1191 1248 1249 1250 1254 1272 1273 1449 1522 1604 1658 1731 1762
 1764 1783 1791 1798 1802]


# Moving

In [9]:
import pandas as pd

# Step 1: Open the CSV and count occurrences of 'yes' and 'no' in the 'Moving' column
def count_moving_column_values(csv_file):
    # Read the CSV file into a DataFrame
    df = pd.read_csv(csv_file)
    
    # Ensure the 'Moving' column exists
    if 'Moving' not in df.columns:
        print(f"Error: 'Moving' column not found in {csv_file}")
        return
    
    # Count the occurrences of 'yes' and 'no'
    yes_count = df['Moving'].str.lower().value_counts().get('yes', 0)
    no_count = df['Moving'].str.lower().value_counts().get('no', 0)

    # Return the counts
    return yes_count, no_count

# Main function to execute the script
def main():
    csv_file = 'LVL2_Full.csv'  # Replace with the path to your .csv file

    # Get the counts of 'yes' and 'no' in the 'Moving' column
    yes_count, no_count = count_moving_column_values(csv_file)

    # Output the results
    print(f"'Yes' count in 'Moving' column: {yes_count}")
    print(f"'No' count in 'Moving' column: {no_count}")


main()

'Yes' count in 'Moving' column: 95
'No' count in 'Moving' column: 1511
